# Machine Translation From Yoruba To English

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from string import digits

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


import re
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Bidirectional
from tensorflow.keras.layers import Activation, dot, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize



In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2084559632491640539
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6597931449
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17464277316235132598
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [79]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sub = pd.read_csv('SampleSubmission.csv')
train.head()

,ID,Yoruba,English
0,ID_AAJEQLCz,A ṣètò Ìgbìmọ̀ Tó Ń Ṣètò Ìrànwọ́ Nígbà Àjálù l...,A Disaster Relief Committee was formed to orga...
1,ID_AASNedba,"Ìrọ̀lẹ́ May 22, 2018 ni wọ́n fàṣẹ ọba mú Arákù...",Brother Solovyev was arrested on the evening o...
2,ID_AAeQrhMq,Iléeṣẹ́ Creative Commons náà,Creative Commons the Organization
3,ID_AAxlMgPP,"Pè̩lú Egypt, Morocco àti Tunisia tí wó̩n ti lo...","With Egypt, Morocco and Tunisia out of the Wor..."
4,ID_ABKuMKSx,Adájọ́ àgbà lórílẹ̀ èdè Náíjíríà (Attorney Gen...,"The Attorney General of the Federation, Justic..."


In [80]:
test.head()

,ID,Yoruba
0,ID_AAAitMaH,"Nínú ìpè kan lẹ́yìn ìgbà náà, wọ́n sọ fún aṣoj..."
1,ID_AAKKdQwr,Nítorí kò sí nǹkan tí ọkùnrin ò lè ṣe láì náán...
2,ID_ABgAyEOp,Bí i kó pariwo. Kí ó kígbe mọ́ ẹ?
3,ID_ACFgfKQs,"Tí ó ń lé e lọ sọ́nà etí odò Akókurà, tí ó bẹ̀..."
4,ID_ACNPmlhf,Èṣúńiyì mọ̀ iṣẹ́ rẹ̀ dunjú. Màmá tirí bí ó ṣe ...


In [81]:
sub.head()

,ID,English
0,ID_AAAitMaH,NaN
1,ID_AAKKdQwr,NaN
2,ID_ABgAyEOp,NaN
3,ID_ACFgfKQs,NaN
4,ID_ACNPmlhf,NaN


In [82]:
train.shape

(10054, 3)

In [83]:
# train_X,train_y = train_df.loc[:,['ID','Yoruba']], train_df.loc[:,'English']
train.Yoruba = train.Yoruba.apply(lambda x: x.lower())
train.English = train.English.apply(lambda x: x.lower())
train[train.ID =='ID_WZXuYKCN']

,ID,Yoruba,English
4379,ID_WZXuYKCN,5. ṣiṣẹ́ ọ̀rọ̀-ìfiwọlé tó l'ágbára,5.creating strong passwords


In [84]:
# train_X.head()
exclude = set(string.punctuation)
train.Yoruba = train.Yoruba.apply(lambda x: "".join(("".join([" "if ch in exclude else ch for ch in x]))))
train.English = train.English.apply(lambda x: "".join(("".join([" "if ch in exclude else ch for ch in x]))))
# " ".join("".join([" " if ch in exclude else ch for ch in x)                                    
train[train.ID =='ID_WZXuYKCN']

,ID,Yoruba,English
4379,ID_WZXuYKCN,5 ṣiṣẹ́ ọ̀rọ̀ ìfiwọlé tó l ágbára,5 creating strong passwords


In [85]:
# train.English = train.English.apply(lambda x: x.strip())
# train.Yoruba = train.Yoruba.apply(lambda x: x.strip())

# train.English = train.English.apply(lambda x: re.sub(" +", " ", x))
# train.Yoruba = train.Yoruba.apply(lambda x: re.sub(" +", " ", x))

# train.English = train.English.apply(lambda x: re.sub("\?\?", '', x))
# train.Yoruba = train.Yoruba.apply(lambda x: re.sub("\?\?", '', x))

train[train.ID =='ID_WZXuYKCN']

,ID,Yoruba,English
4379,ID_WZXuYKCN,5 ṣiṣẹ́ ọ̀rọ̀ ìfiwọlé tó l ágbára,5 creating strong passwords


In [86]:
train.columns
train.sample(10)

,ID,Yoruba,English
2373,ID_MCjJBkua,fífẹ́ la fẹ́ ẹ̀fọ́ tí à ń pèé ní ọ̀rẹ́ ẹ̀kọ ti...,it is only because one loves spinach that one ...
927,ID_EgxsITdG,ilé ẹjọ́ echr sì sọ pé wọ́n máa rí i dájú pé à...,the echr has indicated that it will closely mo...
5992,ID_erTpSsFw,orílẹ̀ èdè amẹ́ríkà la ti kọ́kọ́ ṣe ilé ẹ̀kọ́ ...,the pioneer service school began in the united...
3085,ID_PhomklDI,ní december 29 2018 ìjì líle tí wọ́n pè ní ...,on december 29 2018 tropical depression usm...
6041,ID_fDTiLvhs,cc tí í ṣe iléeṣẹ́ àìlérèlórí tí ó fi ìgbà ka...,cc the nonprofit organization once housed wit...
3950,ID_UHuJrnRe,ẹ̀rọ òṣùnwọ̀n ìgbóná,the thermometer
4286,ID_VsghGfft,a bá àwọn arákùnrin àti arábìnrin wa ní azerba...,we rejoice with our brothers and sisters in az...
7955,ID_ovyyGLJw,àwọn ẹlẹ́rìí jèhófà ti lómìnira ẹ̀sìn ní mòǹgó...,victory for religious freedom in mongolia jeh...
9833,ID_zCKNibjD,omi ló dànù agbè ò fọ́,it is the water that is spilled the water gou...
6165,ID_fmweGFXY,gẹ́gẹ́ bí alákòóso ikọ̀ tó ń sojú cameroon nín...,the country’s team manager victor agbor nso ...


In [87]:
# train_y.head()
vocab_yoruba =set()
for sent in train.Yoruba:
    for word in sent.split():
        if word not in vocab_yoruba:
            vocab_yoruba.add(word)
        

vocab_english = set()
for sent in train.English:
    for word in sent.split():
        if word not in vocab_english:
            vocab_english.add(word)
            

In [88]:
length_list = []
for i in train.Yoruba:
    length_list.append(len(i.split(' ')))
max_length_yoruba = np.max(length_list)
max_length_yoruba, np.average(length_list)

(242, 22.788840262582056)

In [89]:
length_list = []
for i in train.English:
    length_list.append(len(i.split(' ')))
max_length_english = np.max(length_list)
max_length_english, np.average(length_list)

(203, 19.828227571115974)

In [90]:
input_words = sorted(list(vocab_yoruba))
target_words = sorted(list(vocab_english))
num_encoder_tokens = len(vocab_yoruba)
num_decoder_tokens = len(vocab_english)
num_encoder_tokens, num_decoder_tokens

(15466, 15884)

In [97]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [99]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [100]:
train = shuffle(train)
train.head()

,ID,Yoruba,English
7237,ID_lXsueqsR,àwa ẹlẹ́rìí jèhófà ti pe ẹjọ́ kòtẹ́milọ́rùn ló...,jehovah’s witnesses are appealing this case
9886,ID_zNdOGOeK,“mo ṣe é tán” ló níyì a kì í dúpẹ́ aláṣekù,“i have completed the job” is what deserves pr...
364,ID_BkKlhnpi,atanilólobó báńkì àgbáyé kan ni,a world bank informant stated
5373,ID_bkIXnxvr,ìyàwó ṣe ọ̀ràn kan tán ọkọ ẹ̀ ẹ́ ṣe ọ̀ràn an ...,the wife has done the unpardonable her husban...
7765,ID_oEicxvRm,ó dá ẹ lójú pé kò sí ẹnì kankan nínú ilé yìí,are you sure there is no body in this house


In [102]:
yoruba = list(train.Yoruba)
english = list(train.English)

x = []
y = []

for i in range(len(yoruba)):
    x.append(str(yoruba[i]))
    y.append(str(english[i]))

x = np.array(x)
y = np.array(y)
len(x), len(y)

(10054, 10054)

In [103]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
X_train.shape, X_test.shape

((8043,), (2011,))

# Data Preprocessing

## Tokenize

In [221]:
def tokenize(x):
    """
    Tokenize x
    param x: pandas series of sentences/strings to be tokenized
    return: tokenized x data
    """
    sentences = x.tolist()
    tokenizer = Tokenizer(char_level=False)
    tokenizer.fit_on_texts(x)
#     word_index = tokenizer.word_index
    return tokenizer.texts_to_sequences(x)

In [222]:
# train_X['Yoruba'] = train_X.apply(lambda row:tokenize_Yoruba(row['Yoruba']), axis = 1) 
train_X['Yoruba'] = tokenize(train_X['Yoruba'])

In [223]:
train_X.head()

,ID,Yoruba
0,ID_AAJEQLCz,"[12, 192, 102, 4, 5, 192, 227, 74, 216, 11, 40..."
1,ID_AASNedba,"[2345, 520, 873, 204, 15, 7, 1964, 610, 55, 59..."
2,ID_AAeQrhMq,"[553, 510, 459, 14]"
3,ID_AAxlMgPP,"[697, 1291, 1226, 17, 2682, 2, 924, 10, 4726, ..."
4,ID_ABKuMKSx,"[495, 200, 78, 25, 7043, 7044, 3132, 345, 521,..."


## Padding

In [225]:
def pad(x, length=None):
    """
    Pad x
    param x: List of sequences.
    param length: Length to pad the sequence to, If None use length of longest sequence in x
    return: Padded numpy array of sequences

    """
    
    if length is None:
        length = max([len(sentence) for sentence in x])
        print(length)
        
    return pad_sequences(x, maxlen=length, padding='post', value=0).tolist()

In [226]:
train_X['Yoruba'] = pad(train_X['Yoruba'])
train_X.head()
# length = None
# x = train_X['Yoruba']
# if length is None:
#         length = max([len(sentence) for sentence in x])
# train_X['Yourba'] = pad_sequences(train_X.Yoruba, maxlen=length, padding='post', value=0).tolist()

226


,ID,Yoruba
0,ID_AAJEQLCz,"[12, 192, 102, 4, 5, 192, 227, 74, 216, 11, 40..."
1,ID_AASNedba,"[2345, 520, 873, 204, 15, 7, 1964, 610, 55, 59..."
2,ID_AAeQrhMq,"[553, 510, 459, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,ID_AAxlMgPP,"[697, 1291, 1226, 17, 2682, 2, 924, 10, 4726, ..."
4,ID_ABKuMKSx,"[495, 200, 78, 25, 7043, 7044, 3132, 345, 521,..."


In [73]:
# (train_lang, valid_lang,train_labels,valid_labels) = train_test_split(train_X,
#                                                                     train_y,train_size = 0.8,random_state = 0)

In [74]:
train_lang

,ID,Yoruba
2560,ID_NBoLPZLV,"[135, 230, 20, 1251, 2, 1, 62, 45, 28, 437, 10..."
7972,ID_pBOTGAWL,"[7, 20, 42, 115, 1663, 158, 307, 306]"
2406,ID_MKYPuSzg,"[168, 51, 23, 49, 241, 129]"
2782,ID_OGkTbtPH,"[3, 915, 1784, 162, 152, 258, 33, 235, 157, 49..."
8649,ID_shPghfmD,"[11, 743, 159, 627, 162, 15, 378, 4, 2680, 4, ..."
...,...,...
9225,ID_vgRSIdjA,"[95, 124, 30, 3, 529, 10, 5, 192, 227, 246, 2,..."
4859,ID_ZCpYPuTO,"[11149, 1, 65, 547, 41, 25, 2934, 38, 18, 11, ..."
3264,ID_QfHQSlhY,"[1142, 6, 5, 218]"
9845,ID_zEaCdxEJ,"[558, 15331]"
